# Network Flows Lab

The goal of this lab is to gain some experience with applying network flows to existing problems. 

The following problem is adapted from [CSES 1711](https://cses.fi/problemset/task/1711):

Before beginning, the following cell contains any dependencies you may need for this assignment. 


In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ortools.graph.python import max_flow

*[Ortools](https://or-tools.github.io/docs/pdoc/ortools/graph/python/max_flow.html) is the library that we will be using to help solve the network flows problems.

## Background Info
A game consists of 6 rooms and 7 teleporters. At the beginning of the day, you start in room 0 and have to reach room 5. 

You can use each teleporter at most once during the game. How many days can you play if you choose your routes optimally?

The lines below represent the connections between the rooms. Each line has two integers: $a$ and $b$. There is a teleporter from room $a$ to room $b$.
0 1 \
0 2 \
1 5 \
2 3 \
2 4 \
3 5 \
4 5 

## Questions
a. Determine the maximum number of days you can play the game. \
b. Write out the route descriptions; this can be any valid solution.


## Graph Setup

First, let's set up our network flow problem in a format that's more familiar: nodes and edges. 

Each room represents a node and each teleporter represents an edge. Given that knowledge, fill out the `start_nodes` and `end_nodes` numpy arrays.

In [ ]:
start_nodes = np.array([]) # Replace with your code
end_nodes = np.array([]) # Replace with your code

Now that we have populated our start and end nodes, we have to think about the capacities for each edge.

In this version of the problem, remember that each teleporter can only be used once.

In [ ]:
capacities = np.array([]) # Replace with your code

The cell below populates the information you have collected into a graph:

In [ ]:
# Create an edgelist dataframe
edgelist_df = pd.DataFrame({'node1':start_nodes, 'node2':end_nodes,
                            'weights':capacities
                           })
edgelist_df

# Create a graph and add edges
g = nx.DiGraph()
for i, elrow in edgelist_df.iterrows():
    g.add_edge(elrow[0], elrow[1], weight=elrow[2])
g.edges(data=True)

# Add nodes to graph
g.add_node(0, pos=(0, 1))
g.add_node(1, pos=(0, 0))
g.add_node(2, pos=(1, 1))
g.add_node(3, pos=(1, 0))
g.add_node(4, pos=(2, 1))
g.add_node(5, pos=(2, 0))
g.nodes(data=True)

# Display graph
pos = nx.get_node_attributes(g,'pos')
weight = nx.get_edge_attributes(g,'weight') 
nx.draw(g,pos, with_labels=True)
nx.draw_networkx_edge_labels(g,pos,edge_labels = weight)
plt.show()

## A. Determine Maximum \# of Days You Can Play
The maximum # of days you can play the game can be represented by the maximum flow of the graph we just created. 

Since you can only use each teleporter once, each path taken cannot share edges. These are called **disjoint paths**. 

The maximum # of disjoint paths between two nodes is equal to the maximum flow according to the max flow min cut theorem we went over in class (NF Day 1).

In the following cell, use a simple max flow solver to calculate the maximum flow of the graph you created.

Helpful information: 
- Ortools documentation, including functions you will need, can be found [here](https://or-tools.github.io/docs/pdoc/ortools/graph/python/max_flow.html)
- An arc consists of a start node, an end node, and a capacity (or flow) from the start to the end node

In [ ]:
# Instantiate a SimpleMaxFlow solver.
smf = max_flow.SimpleMaxFlow()

# Add arcs with capacities to the solver; this can be done either in bulk or one by one.
all_arcs = None

# Source & sink node; add in your code
source = None
sink = None

# Use simple max flow's solver & print your results

## B. Route Descriptions
To ge the route descriptions, we need to extract the disjoint paths from our solution.

First, let's get the flow values for each arc: 

In [ ]:
solution_flows = smf.flows(all_arcs)

Next, we need to identify the disjoint paths by checking the flow values. This can be done by traversing the graph starting from the source and following each path, keeping track of the arcs we've already used 

Start with a given node and iterate thorugh the graph until you have reached the final node. 

Helpful information:
- `smf.tail(arc)` should give a start_node
- `smf.head(arc)` should give a corresponding end_node
- You can add to a set with `set.add(item)`
- Remember to append to your path where you have visited!


In [ ]:
used_arcs = set() # data structure to store used paths
def find_path(start_node):
    path = []
    # Iterate through the graph from the starting node to the sink
    # For each arc, check if there is a solution that is not in used_arcs
    # If there is, add the start and end nodes to the path & the arc itself to used arcs
    return path

You've calculated paths at this point, but now, you need to print your results. 
This is also a good way to check whether your results are logical. 

*Repeting nodes (ex 1 -> 2 -> 2 -> 3) is fine to represent 1 -> 2 -> 3.

In [ ]:
disjoint_paths = []
# Iterate through the results of your optimal flow
# Add the path to disjointed_paths

# Prints the paths
for path in disjoint_paths:
    print (" -> ".join(map(str, path)))